# Notebook Contents

In this notebook the idea os to generate the necessary enviornment lattices for the multi-agent system which are based on calculation of Eucledian Distances.
Hence the lattices which are generated in this notebook include.
1. Distance form Facades (NSEW)
2. Distance from Roof / FLoor 
3. Quiteness lattice (Distance from source of sound) 

The steps followed in the notebook inculde :
1. Load all the assets required for simulation
2. Load the excel files showing the points of interest (In Quiteness Lattice it is the co-ordinates of the Sound source) 
3. Calculate the distances using Scipy https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html
4. Vizualise the lattices on the building
5. Save the lattices as pickle objects


# Initilization

In [1]:
import os
import sys
import topogenesis as tg
import pickle
import pyvista as pv
import pandas as pd
import trimesh as tm
import scipy as sc
from scipy.spatial import distance
import itertools
from itertools import cycle 
import numpy as np
import numpy.ma as ma
np.random.seed(0)
np.set_printoptions(threshold=sys.maxsize)
from ladybug.sunpath import Sunpath

In [74]:
#Design Details
Self_development_plots_path = os.path.relpath('Site_self_development_plots.obj')
Self_development_backyards_path = os.path.relpath('Site_self_development_backyards.obj')
Site_buildings_path = os.path.relpath('Site_buildings.obj')
Site_green_areas_path = os.path.relpath('Site_green_areas.obj')
Site_roads_path = os.path.relpath('Site_base_roads.obj')
Site_context_shading_path= os.path.relpath('Site_surrounding_buildings_for_shading.obj') 
Offices_block= os.path.relpath('Site_office_blocks.obj') 

# Site details
Site_base_path = os.path.relpath('Site_base_block.obj')
Site_surrounding_buildings_path = os.path.relpath('Site_surrounding_buildings.obj')
Site_water_bodies_path = os.path.relpath('Site_water_bodies.obj')
Site_roads_path = os.path.relpath('Site_roads.obj')
Site_other_buildings_path = os.path.relpath('Site_other_buildings.obj')

# load the mesh from file
# Design elements
Self_development_plots_mesh = tm.load(Self_development_plots_path)
Self_development_backyards_mesh = tm.load(Self_development_backyards_path)
Site_building_mesh = tm.load(Site_buildings_path)
Site_green_areas_mesh = tm.load(Site_green_areas_path)
Site_roads_mesh = tm.load(Site_roads_path)
Site_context_shading_mesh = tm.load(Site_context_shading_path)
Site_offices_zone_mesh = tm.load(Offices_block)

#Site elements
Site_base_mesh = tm.load(Site_base_path)
Site_surrounding_buildings_mesh = tm.load(Site_surrounding_buildings_path)
Site_water_bodies_mesh = tm.load(Site_water_bodies_path)
Site_roads_mesh = tm.load(Site_roads_path)
Site_other_buildings_mesh = tm.load(Site_other_buildings_path)


# Check if the mesh is watertight
#print(envelope_mesh.is_watertight)
#print(context_mesh.is_watertight)

faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!


# Define Points for Quiteness

## import from Rhino 

In [3]:
# Csv to points 
Complete_file = pd.read_excel('Quiteness_points.xlsx', sheet_name=0,engine='openpyxl',header = None )
design_criterias= list(Complete_file.head(n=0))
array_excel = Complete_file.to_numpy()

In [4]:
# Csv to points 
Complete_file_GA = pd.read_excel('Points_of_interest.xlsx', sheet_name=0,engine='openpyxl',header = None )
design_criterias_GA= list(Complete_file_GA.head(n=0))
array_excel_GA = Complete_file_GA.to_numpy()

In [5]:
Noise_sources = array_excel.T
Noise_sources_GA = array_excel_GA.T

# Define Points for Closeness Facades

In [6]:
len(Noise_sources_GA)

125

# Vizualise the meshes and points

In [7]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=True)

# adding the meshes
# Design meshes
p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#f4acb7')
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd',opacity= 0.5)

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_surrounding_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
#p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')

p.add_points( Noise_sources, color='#e63946')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(1563.6562028391934, 1809.440031574545, 1616.2765183909512),
 (-142.0987091064453, 103.68511962890625, -89.4783935546875),
 (0.0, 0.0, 1.0)]

In [76]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=True)

# adding the meshes
# Design meshes
p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#f4acb7',opacity = 0.1)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_offices_zone_mesh), color='#bdb2ff')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_surrounding_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
##p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')

p.add_points( Noise_sources_GA, color='#e63946')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(1563.6562028391934, 1809.440031574545, 1616.2765183909512),
 (-142.0987091064453, 103.68511962890625, -89.4783935546875),
 (0.0, 0.0, 1.0)]

# Import lattice

In [9]:
# loading the lattice from csv
lattice_path = os.path.relpath('voxelized_envelope_3m_voxel_size_for_offices.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)

In [10]:
envelope_lattice.size

4123

# Visualize the Context Mesh + Envelope Lattice

In [90]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=True)
p.camera_position = [(-30.918138503987137, 252.13468433505227, 141.27150258463084),
 (-132.52727934148325, 29.061346168534897, -31.80320438629297),
 (-0.1996427382461422, -0.5422754627817726, 0.8161373043369582)]
# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes

# Design meshes
p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_offices_zone_mesh), color='#bdb2ff',opacity= 0.8)
#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_surrounding_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
# plotting
p.add_points( Noise_sources_GA, color='#e63946')
p.show(use_ipyvtk=True,screenshot=' Closeness_to_grren_Areas_problem.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(-30.918138503987137, 252.13468433505227, 141.27150258463084),
 (-132.52727934148325, 29.061346168534897, -31.80320438629297),
 (-0.1996427382461422, -0.5422754627817726, 0.8161373043369582)]

# Find the voxel Eucledian coordinates

In [12]:
# convert to lattice
init_lattice = envelope_lattice +1
availability_lattice_voxels = tg.to_lattice(init_lattice, init_lattice)
voxel_coordinates= availability_lattice_voxels.centroids


In [13]:
flattened_lattice = envelope_lattice.flatten()

# Find distances for Quiteness 

The distance from each source of sound is calculated and the minimum distance from them is choosen

In [14]:
Eucledian_distance = sc.spatial.distance.cdist(# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=True)
p.camera_position = [(-30.918138503987137, 252.13468433505227, 141.27150258463084),
 (-132.52727934148325, 29.061346168534897, -31.80320438629297),
 (-0.1996427382461422, -0.5422754627817726, 0.8161373043369582)]
# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes

# Design meshes
p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_offices_zone_mesh), color='#bdb2ff',opacity= 0.8)
#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_surrounding_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
# plotting
p.add_points( Noise_sources_GA, color='#e63946')
p.show(use_ipyvtk=True,screenshot=' Closeness_to_grren_Areas_problem.png'),voxel_coordinates)

In [15]:
noise_from_each_source = Eucledian_distance.T

In [16]:
Average_quiteness_indexing= np.argmin(noise_from_each_source, axis=1)

In [17]:
Average_quiteness_values = []
for branch,index in zip(noise_from_each_source,Average_quiteness_indexing):
    Average_quiteness_values.append(branch[index])
    

In [18]:
Quiteeness_lattice_padded= np.array([num if boolean else 0 for boolean, num in zip(flattened_lattice, cycle(Average_quiteness_values))])

In [19]:
padded_array = np.array(Quiteeness_lattice_padded)

In [20]:
Quiteness_lattice_np = Quiteeness_lattice_padded.reshape(envelope_lattice.shape)

In [21]:
Quiteness_lattice =tg.to_lattice(Quiteness_lattice_np, Quiteness_lattice_np.shape)

# Find distances for Green Areas

In [93]:
Eucledian_distance_GA = sc.spatial.distance.cdist(Noise_sources_GA,voxel_coordinates)

In [94]:
noise_from_each_source_GA = Eucledian_distance_GA.T

In [95]:
Average_quiteness_indexing_GA= np.argmin(noise_from_each_source_GA, axis=1)

In [96]:
Average_quiteness_values_GA = []
for branch,index in zip(noise_from_each_source_GA,Average_quiteness_indexing_GA):
    Average_quiteness_values_GA.append(branch[index])
    

In [98]:
Quiteeness_lattice_padded_GA= np.array([num if boolean else 0 for boolean, num in zip(flattened_lattice, cycle(Average_quiteness_values_GA))])

In [99]:
padded_array_GA = np.array(Quiteeness_lattice_padded_GA)

In [100]:
Quiteness_lattice_np_GA = Quiteeness_lattice_padded_GA.reshape(envelope_lattice.shape)

In [101]:
Quiteness_lattice_GA =tg.to_lattice(Quiteness_lattice_np_GA, Quiteness_lattice_np_GA.shape)

# Find distances for Facade calculations

In [22]:
Y_coordinates= voxel_coordinates.T[1].flatten()

In [23]:
X_coordinates= voxel_coordinates.T[0].flatten()

In [24]:
Building_classification_north_south_facades =[]

for center in Y_coordinates:
    if center >= 18 and center <= 30:
        Building_classification_north_south_facades.append(center-17)
        #print("1st")
    elif center >= 40 and center <= 82:
        Building_classification_north_south_facades.append(center-41)
       # print("2nd")
    elif center >= 82:
        Building_classification_north_south_facades.append(center-83)
        #print("3rd")
    else:
        Building_classification_north_south_facades.append(center)
    

In [25]:
Building_classification_east_west_facades =[]

for center,value in zip(Y_coordinates,X_coordinates):
    if center <= 90:
        Building_classification_east_west_facades.append(10-value)
        #print("1st")
    elif center >= 91:
        Building_classification_east_west_facades.append(130-value)
        #print("2nd")

# Facade Distances to North Facade

In [26]:
Distance_inverse_y_axis_north = [i for i in reversed(Building_classification_north_south_facades)]

In [27]:
Distance_north_facade_full_lattice= np.tile(Distance_inverse_y_axis_north,352)

In [28]:
North_facade_lattice_padded= np.array([num if boolean else 0 for boolean, num in zip(flattened_lattice, cycle(Distance_inverse_y_axis_north))])

In [29]:
North_facade_array = np.array(North_facade_lattice_padded)

In [30]:
north_facade_lattice_np = North_facade_array.reshape(envelope_lattice.shape)

In [31]:
North_facade_lattice =tg.to_lattice(north_facade_lattice_np, north_facade_lattice_np.shape)

# Facade Distances to South Facade

In [32]:
South_facade_lattice_padded= np.array([num if boolean else 0 for boolean, num in zip(flattened_lattice, cycle(Building_classification_north_south_facades))])

In [33]:
Souh_facade_array = np.array(South_facade_lattice_padded)

In [34]:
South_facade_lattice_np = Souh_facade_array.reshape(envelope_lattice.shape)

In [35]:
South_facade_lattice =tg.to_lattice(South_facade_lattice_np, South_facade_lattice_np.shape)

# Facade Distances to West Facade

In [36]:
West_facade_lattice_padded= np.array([num if boolean else 0 for boolean, num in zip(flattened_lattice, cycle(Building_classification_east_west_facades))])

In [37]:
West_facade_array = np.array(West_facade_lattice_padded)

In [38]:
West_facade_lattice_np = West_facade_array.reshape(envelope_lattice.shape)

In [39]:
West_facade_lattice =tg.to_lattice(West_facade_lattice_np, West_facade_lattice_np.shape)

# Facade Distances to East Facade

In [40]:
Distance_inverse_y_axis_west = [i for i in reversed(Building_classification_east_west_facades)]

In [41]:
West_facade_lattice_padded= np.array([num if boolean else 0 for boolean, num in zip(flattened_lattice, cycle(Distance_inverse_y_axis_west))])

In [42]:
East_facade_array = np.array(West_facade_lattice_padded)

In [43]:
East_facade_lattice_np = East_facade_array.reshape(envelope_lattice.shape)

In [44]:
East_facade_lattice =tg.to_lattice(East_facade_lattice_np, East_facade_lattice_np.shape)

# Find distances from Ground calculations

In [45]:
Distance_from_ground_complete_lattice = voxel_coordinates.T[2]

In [46]:
envelope_lattice.size

4123

In [47]:
Distance_from_ground_lattice_padded= np.array([num if boolean else 0 for boolean, num in zip(flattened_lattice, cycle(Distance_from_ground_complete_lattice))])

In [48]:
padded_ground_distance_array = np.array(Distance_from_ground_lattice_padded)

In [49]:
padded_ground_distance_array_np = padded_ground_distance_array.reshape(envelope_lattice.shape)

In [50]:
Distance_from_ground = tg.to_lattice(padded_ground_distance_array_np, padded_ground_distance_array_np.shape)

In [51]:
envelope_lattice.shape

(31, 7, 19)

# Find distances from Roof calculations

In [52]:
all_heights_reversed= [i for i in reversed(voxel_coordinates.T[2])]

In [53]:
Distance_from_roof_complete_lattice = np.tile(all_heights_reversed, int(envelope_lattice.size/11))

In [54]:
Distance_from_roof_lattice_padded= np.array([num if boolean else 0 for boolean, num in zip(flattened_lattice, cycle(Distance_from_roof_complete_lattice))])

In [55]:
padded_roof_distance_array = np.array(Distance_from_roof_lattice_padded)

In [56]:
padded_roof_distance_array_np =  padded_roof_distance_array.reshape(envelope_lattice.shape)

In [57]:
Distance_from_roof = tg.to_lattice(padded_roof_distance_array_np, padded_roof_distance_array_np.shape)

# Visualize the Quiteness Lattice

In [58]:
###### initiating the plotter
p = pv.Plotter(notebook=True)
# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid envelope_lattice: shape because we want to inject our values
grid.dimensions = envelope_lattice.shape
# The bottom left corner of the data set
grid.origin = envelope_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = envelope_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Quiteness"] = Quiteness_lattice.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
#p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_context_shading_mesh), color='#cdb4db')
    
# adding the volume

opacity = [0, 0.75, 0, 0.75, 1.0] 
clim = [0, 100]
p.add_volume(grid, cmap="magma", clim=clim,
             opacity=opacity, opacity_unit_distance=5,)
p.add_points( Noise_sources, color='#e63946')
# plotting
#p.camera_position = [(87, 269, 373), (-300, 70, -0), (0, 0, 1)]
#p.image_depth
p.camera_position = [(-30.918138503987137, 252.13468433505227, 141.27150258463084),
 (-132.52727934148325, 29.061346168534897, -31.80320438629297),
 (-0.1996427382461422, -0.5422754627817726, 0.8161373043369582)]
p.show(use_ipyvtk=True,screenshot='Quiteness_lattice.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(-30.918138503987137, 252.13468433505227, 141.27150258463084),
 (-132.52727934148325, 29.061346168534897, -31.80320438629297),
 (-0.1996427382461422, -0.5422754627817726, 0.8161373043369582)]

# Viz Closeness to Green areas

In [105]:
###### initiating the plotter
p = pv.Plotter(notebook=True)
# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid envelope_lattice: shape because we want to inject our values
grid.dimensions = envelope_lattice.shape
# The bottom left corner of the data set
grid.origin = envelope_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = envelope_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Closeness to Green Areas"] = Quiteness_lattice_GA.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
#p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_context_shading_mesh), color='#cdb4db')
    
# adding the volume

opacity = [0, 0.75, 0, 0.75, 1.0] 
clim = [0, 100]
p.add_volume(grid, cmap="magma", clim=clim,
             opacity=opacity, opacity_unit_distance=5,)
#p.add_points( Noise_sources, color='#e63946')
# plotting
#p.camera_position = [(87, 269, 373), (-300, 70, -0), (0, 0, 1)]
#p.image_depth
p.camera_position = [(-30.918138503987137, 252.13468433505227, 141.27150258463084),
 (-132.52727934148325, 29.061346168534897, -31.80320438629297),
 (-0.1996427382461422, -0.5422754627817726, 0.8161373043369582)]
p.add_points( Noise_sources_GA, color='#9bf6ff')
p.show(use_ipyvtk=True,screenshot=' Closeness_to_grren_Areas_viz.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(-30.918138503987137, 252.13468433505227, 141.27150258463084),
 (-132.52727934148325, 29.061346168534897, -31.80320438629297),
 (-0.1996427382461422, -0.5422754627817726, 0.8161373043369582)]

In [59]:
p.camera_position

[(-30.918138503987137, 252.13468433505227, 141.27150258463084),
 (-132.52727934148325, 29.061346168534897, -31.80320438629297),
 (-0.1996427382461422, -0.5422754627817726, 0.8161373043369582)]

# Visualize the N Facade Lattice

In [60]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid envelope_lattice: shape because we want to inject our values
grid.dimensions = envelope_lattice.shape
# The bottom left corner of the data set
grid.origin = envelope_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = envelope_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Distance from North Facade"] = North_facade_lattice.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
#p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_context_shading_mesh), color='#cdb4db')
    
# adding the volume

opacity = [0, 0.75, 0.7, 0.75, 0.8]
clim = [0, 100]
p.add_volume(grid, cmap="viridis", opacity=opacity, shade=False)
# plotting
p.camera_position = [(-30.918138503987137, 252.13468433505227, 141.27150258463084),
 (-132.52727934148325, 29.061346168534897, -31.80320438629297),
 (-0.1996427382461422, -0.5422754627817726, 0.8161373043369582)]
p.show(use_ipyvtk=True,screenshot='Distance_from_North_Facade.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(-30.918138503987137, 252.13468433505227, 141.27150258463084),
 (-132.52727934148325, 29.061346168534897, -31.80320438629297),
 (-0.1996427382461422, -0.5422754627817726, 0.8161373043369582)]

# Visualize the S Facade Lattice

In [61]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid envelope_lattice: shape because we want to inject our values
grid.dimensions = envelope_lattice.shape
# The bottom left corner of the data set
grid.origin = envelope_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = envelope_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Distance from South Facade"] = South_facade_lattice.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
#p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_context_shading_mesh), color='#cdb4db')
    
# adding the volume

opacity = [0, 0.75, 0.7, 0.75, 0.8]
clim = [0, 100]
p.add_volume(grid, cmap="viridis", opacity=opacity, shade=False)
# plotting
p.camera_position = [(-30.918138503987137, 252.13468433505227, 141.27150258463084),
 (-132.52727934148325, 29.061346168534897, -31.80320438629297),
 (-0.1996427382461422, -0.5422754627817726, 0.8161373043369582)]
p.show(use_ipyvtk=True,screenshot='Distance_from_South_Facade.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(-30.918138503987137, 252.13468433505227, 141.27150258463084),
 (-132.52727934148325, 29.061346168534897, -31.80320438629297),
 (-0.1996427382461422, -0.5422754627817726, 0.8161373043369582)]

# Visualize the E Facade Lattice

In [62]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid envelope_lattice: shape because we want to inject our values
grid.dimensions = envelope_lattice.shape
# The bottom left corner of the data set
grid.origin = envelope_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = envelope_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Distance from East Facade"] = East_facade_lattice.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
#p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_context_shading_mesh), color='#cdb4db')
    
# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6])*1.5
opacity = [0, 0.75, 0, 0.75, 1.0]
clim = [0, 100]
p.add_volume(grid, cmap="viridis", opacity=opacity, shade=False)
p.camera_position = [(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154929, -0.31327193009210785, 0.8220766014501246)]
# plotting
p.show(use_ipyvtk=True,screenshot='Distance_from_East_Facade.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154928, -0.3132719300921078, 0.8220766014501244)]

# Visualize the W Facade Lattice

In [63]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid envelope_lattice: shape because we want to inject our values
grid.dimensions = envelope_lattice.shape
# The bottom left corner of the data set
grid.origin = envelope_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = envelope_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Distance from West Facade"] = West_facade_lattice.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
#p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_context_shading_mesh), color='#cdb4db')
    
# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6])*1.5
opacity = [0, 0.75, 0, 0.75, 1.0]
clim = [0, 100]
p.add_volume(grid, cmap="viridis", opacity=opacity, shade=False)
# plotting
p.camera_position = [(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154929, -0.31327193009210785, 0.8220766014501246)]
p.show(use_ipyvtk=True,screenshot='Distance_from_West_Facade.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154928, -0.3132719300921078, 0.8220766014501244)]

# Visualize the Terrace Lattice

In [64]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid envelope_lattice: shape because we want to inject our values
grid.dimensions = envelope_lattice.shape
# The bottom left corner of the data set
grid.origin = envelope_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = envelope_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Distance from Roof"] = Distance_from_roof.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
#p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_context_shading_mesh), color='#cdb4db')
    
# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6])*1.5
opacity = [0, 0.75, 0, 0.75, 1.0]
clim = [0, 100]
p.add_volume(grid, cmap="viridis", opacity=opacity, shade=False)
# plotting
p.camera_position = [(-30.918138503987137, 252.13468433505227, 141.27150258463084),
 (-132.52727934148325, 29.061346168534897, -31.80320438629297),
 (-0.1996427382461422, -0.5422754627817726, 0.8161373043369582)]
p.show(use_ipyvtk=True,screenshot='Distance_from_Roof.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(-30.918138503987137, 252.13468433505227, 141.27150258463084),
 (-132.52727934148325, 29.061346168534897, -31.80320438629297),
 (-0.1996427382461422, -0.5422754627817726, 0.8161373043369582)]

# Visualize the Ground lattice

In [65]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid envelope_lattice: shape because we want to inject our values
grid.dimensions = envelope_lattice.shape
# The bottom left corner of the data set
grid.origin = envelope_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = envelope_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Distance from ground"] = Distance_from_ground.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
#p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_context_shading_mesh), color='#cdb4db')
    
# adding the volume
opacity = [0, 0.75, 0, 0.75, 1.0] 
clim = [0, 100]
p.add_volume(grid, cmap="viridis", opacity=opacity, shade=False)
# plotting
p.camera_position = [(-30.918138503987137, 252.13468433505227, 141.27150258463084),
 (-132.52727934148325, 29.061346168534897, -31.80320438629297),
 (-0.1996427382461422, -0.5422754627817726, 0.8161373043369582)]
p.show(use_ipyvtk=True,screenshot='Distance_from_Ground.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(-30.918138503987137, 252.13468433505227, 141.27150258463084),
 (-132.52727934148325, 29.061346168534897, -31.80320438629297),
 (-0.1996427382461422, -0.5422754627817726, 0.8161373043369582)]

In [66]:
North_facade_lattice

lattice([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
            0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
            0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
            0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
            0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
            0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
            0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
            0.,  0.,  0.,  0.,  0.,  0.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
            0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 16., 16.

# Pickle all Lattices

In [67]:
#Quiteness_mtrx = pickle.dump( Quiteness_lattice, open( "Quiteness_lattice.p", "wb" ) )

In [68]:
#North_facade_mtrx = pickle.dump( North_facade_lattice, open( "North_facade_lattice.p", "wb" ) )

In [69]:
#South_facade_mtrx = pickle.dump( South_facade_lattice, open( "South_facade_lattice.p", "wb" ) )

In [70]:
#East_facade_mtrx = pickle.dump( East_facade_lattice, open( "East_facade_lattice.p", "wb" ) )

In [71]:
#West_facade_mtrx = pickle.dump( West_facade_lattice, open( "West_facade_lattice.p", "wb" ) )

In [72]:
#Terrace_facade_mtrx = pickle.dump( Distance_from_roof, open( "Distance_from_roof.p", "wb" ) )

In [73]:
#Ground_facade_mtrx = pickle.dump( Distance_from_ground, open( "Distance_from_ground.p", "wb" ) )

In [106]:
#Closeness_to_green_areas_mtrx = pickle.dump( Quiteness_lattice_GA, open( "Closeness_to_green_areas.p", "wb" ) )